In [1]:
import importlib

import mysocket as sock

importlib.reload(sock)

<module 'mysocket' from '/home/nghiaht7/data-engineer/public-datasystems-s20/03-Data-Source-stuff/03_20-first/mysocket.py'>

In [2]:
webconn = sock.makeConnection("personal.denison.edu", 80)

In [3]:
lines = ["GET / HTTP/1.1", "Host: personal.denison.edu", ""]

In [4]:
for line in lines:
    sock.sendString(webconn, line)
    sock.sendCRLF(webconn)

In [5]:
s = sock.receiveTillClose(webconn)

In [6]:
s

'HTTP/1.1 200 OK\r\nDate: Fri, 22 Oct 2021 01:49:15 GMT\r\nServer: Apache/2.4.6 (CentOS)\r\nAccept-Ranges: bytes\r\nContent-Length: 464\r\nConnection: close\r\nContent-Type: text/html; charset=iso-8859-1\r\n\r\n<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2//EN">\n<HTML> \n<HEAD>    \n<TITLE>Personal Pages - Denison University</TITLE>\n</HEAD> <BODY> \n<p>&ldquo;Disclaimer: The views and opinions expressed in this service are strictly those of the information provider. The provider assumes full responsibility and liability for the content of this service. The contents of this service have neither been reviewed nor approved by Denison University.&rdquo;</p>\n</BODY> \n</HTML> \n'

In [7]:
webconn.close()

In [8]:
def getMailCode(serverline):
    endpos = serverline.index(" ")
    first = serverline[:endpos]
    num = int(first)
    return num

In [9]:
mailserver = sock.makeConnection("mail.denison.edu", 25)

KeyboardInterrupt: 

In [41]:
s1 = sock.receiveToCRLF(mailserver)
status = getMailCode(s1)
print(repr(s1), status)

'220 smtpint2.cc.denison.edu ESMTP Postfix\r\n' 220


In [ ]:
prefixlines = [
    "HELO 140.141.92.5",
    "MAIL FROM:<bressoud@denison.edu>",
    "RCPT TO:<tcbressoud@gmail.com>",
]

headers = [
    'From: "Dr. B" <bressoud@denison.edu>',
    'To: "Tom" <tcbressoud@gmail.com>',
    "Date: Sun, 29 Mar 2020 12:00:00 -0500",
    "Subject: Test message",
    "",
]

body = """Hello Tom-
This is a test message with four header lines and
five lines in the message body.

Best, Dr. B
"""

for line in prefixlines:
    sock.sendString(mailserver, line)
    sock.sendCRLF(mailserver)
    statusline = sock.receiveToCRLF(mailserver)
    assert getMailCode(statusline) == 250

sock.sendString(mailserver, "DATA")
sock.sendCRLF(mailserver)
statusline = sock.receiveToCRLF(mailserver)
assert getMailCode(statusline) == 354

for headerline in headers:
    sock.sendString(mailserver, headerline)
    sock.sendCRLF(mailserver)

for bodyline in body.split("\n"):
    sock.sendString(mailserver, bodyline)
    sock.sendCRLF(mailserver)

sock.sendString(mailserver, ".")
sock.sendCRLF(mailserver)

statusline = sock.receiveToCRLF(mailserver)
print(repr(statusline))
assert getMailCode(statusline) == 250

In [ ]:
sock.sendString(mailserver, "QUIT")
sock.sendCRLF(mailserver)
statusline = sock.receiveToCRLF(mailserver)
assert getMailCode(statusline) == 221

In [ ]:
mailserver.close()

request_line = ’GET /status/301 HTTP/1.1\r\n’

host_line = ’Host: httpbin.org\r\n’

one_and_done = ’Connection: close\r\n’

empty_line = ’\r\n’

In [ ]:
requestlines = [
    "GET /bookweb/data/basic.html HTTP/1.0",
    "Host: hadoop.mathsci.denison.edu",
    "Connection: close",
    "",
]
request = "\r\n".join(requestlines) + "\r\n"
request

20.14 Suppose we wish to retrieve (GET) a file via HTTP (so port 80)
from datasystems.denison.edu. The resource path of the file is
/data/ind0.json. We wish to use version 1.1 of HTTP and to request that
the connection be closed after a single request/reply exchange. We will need a
header line to satisfy the HTTP 1.1 requirement of a valid Host header. Write a
sequence of code to compose a valid HTTP request as a Python string, and assign
the result to **message**

20.15 Write a sequence of code to establish a connection to the host
datasystems.denison.edu at port 80, to send the string message from
the previous problem to the host, receive the reply from the host until the server
closes the connection, assigning the reply to reply, and close the connection.
Note: if the request is not completely correct, a network connection can wait forever
for a reply that will never come. So if you have difficulty here, double check your
answer to the previous problem.

In [26]:
%load_ext nb_mypy
%nb_mypy On

Version 1.0.2


In [29]:
# 20.14
import re


def printByLine(s) -> None:
    pattern = r"\r\n"
    lines = re.split(pattern, s)
    for line in lines[:-1]:
        real_line = line + "\r\n"
        print(repr(real_line))

In [30]:
protocol = "http"
host = "datasystems.denison.edu"
port = 80
buildURL = lambda s: "{}://{}:{}/{}".format(protocol, host, port, s)

requestlines = [
    "GET /data/ind0.json HTTP/1.1",
    "Host: datasystems.denison.edu",
    "Connection: close",
    "",
]

request = "\r\n".join(requestlines) + "\r\n"
request

'GET /data/ind0.json HTTP/1.1\r\nHost: datasystems.denison.edu\r\nConnection: close\r\n\r\n'

In [31]:
location = host
resource = "/data/ind0.json"
request2 = buildRequest(host, resource)
request2

'GET /data/ind0.json HTTP/1.0\r\nHost: datasystems.denison.edu\r\nConnection: close\r\n\r\n'

In [32]:
printByLine(request)
printByLine(request2)

'GET /data/ind0.json HTTP/1.1\r\n'
'Host: datasystems.denison.edu\r\n'
'Connection: close\r\n'
'\r\n'
'GET /data/ind0.json HTTP/1.0\r\n'
'Host: datasystems.denison.edu\r\n'
'Connection: close\r\n'
'\r\n'


In [20]:
webconn = sock.makeConnection("datasystems.denison.edu", 80)
sock.sendString(webconn, request)
s = sock.receiveTillClose(webconn)

In [21]:
# webconn = sock.makeConnection("hadoop.mathsci.denison.edu", 80)
# for line in requestlines:
#     sock.sendString(webconn, line)
#     sock.sendCRLF(webconn)
# s = sock.receiveTillClose(webconn)

In [22]:
s

'HTTP/1.1 200 OK\r\nDate: Fri, 22 Oct 2021 02:01:50 GMT\r\nServer: Apache/2.4.6 (CentOS)\r\nLast-Modified: Wed, 16 Dec 2020 23:45:42 GMT\r\nETag: "10d-5b69d7922d580"\r\nAccept-Ranges: bytes\r\nContent-Length: 269\r\nConnection: close\r\nContent-Type: application/json\r\n\r\n{"FRA": {"2007": {"pop": 64.02, "gdp": 2657.21}, "2017": {"pop": 66.87, "gdp": 2586.29}}, "GBR": {"2007": {"pop": 61.32, "gdp": 3084.12}, "2017": {"pop": 66.06, "gdp": 2637.87}}, "USA": {"2007": {"pop": 301.23, "gdp": 14451.9}, "2017": {"pop": 325.15, "gdp": 19485.4}}}'

In [24]:
url = buildURL("/data/ind0.json")
url

'http://datasystems.denison.edu:80//data/ind0.json'

In [34]:
protocol = "http"
host = "datasystems.denison.edu"
port = 80
buildURL = lambda s: "{}://{}:{}/{}".format(protocol, host, port, s)

In [101]:
def buildRequest(location: str, resource: str) -> str:
    requestlines = [
        "GET {} HTTP/1.0".format(resource),
        "Host: {}".format(location),
        "Connection: close",
        "",
    ]
    return "\r\n".join(requestlines) + "\r\n"


# def makePOSTRequest(location, resource, body)


def makeRequest(location: str, resource: str) -> str:
    request = buildRequest(location, resource)
    import mysocket as sock

    port = 80
    webconn = sock.makeConnection(location, port)
    sock.sendString(webconn, request)
    s = sock.receiveTillClose(webconn)
    return s


def parseReply(reply: str) -> str:
    return reply[: reply.index("\r\n") + 2]


from typing import Dict, List


def parseHeaders(reply: str) -> Dict[str, str]:
    """
    {'Date': 'Fri, 22 Oct 2021 02:27:36 GMT',
    'Server': 'Apache/2.4.6 (CentOS)',
    'Accept-Ranges': 'bytes',
    'Content-Length': '464',
    'Connection': 'close',
    'Content-Type': 'text/html; charset=iso-8859-1'}"""

    header_and_status_raw = reply[: reply.index("\r\n\r\n")]
    header_raw = header_and_status_raw.split("\r\n")[1:]
    header_list = list(
        map(lambda e: [e.split(" ")[0][:-1], " ".join(e.split(" ")[1:])], header_raw)
    )
    result_header = {e[0]: e[1] for e in header_list}

    return result_header


def parseBody(reply: str):
    return reply[reply.index("\r\n\r\n") + 4 :]

In [102]:
protocol = "http"
host = "datasystems.denison.edu"
port = 80
buildURL = lambda s: "{}://{}:{}/{}".format(protocol, host, port, s)

In [103]:
location = host
resource = "/data/ind0.json"

reply = makeRequest(location=location, resource=resource)
reply

'HTTP/1.1 200 OK\r\nDate: Fri, 22 Oct 2021 03:00:24 GMT\r\nServer: Apache/2.4.6 (CentOS)\r\nLast-Modified: Wed, 16 Dec 2020 23:45:42 GMT\r\nETag: "10d-5b69d7922d580"\r\nAccept-Ranges: bytes\r\nContent-Length: 269\r\nConnection: close\r\nContent-Type: application/json\r\n\r\n{"FRA": {"2007": {"pop": 64.02, "gdp": 2657.21}, "2017": {"pop": 66.87, "gdp": 2586.29}}, "GBR": {"2007": {"pop": 61.32, "gdp": 3084.12}, "2017": {"pop": 66.06, "gdp": 2637.87}}, "USA": {"2007": {"pop": 301.23, "gdp": 14451.9}, "2017": {"pop": 325.15, "gdp": 19485.4}}}'

In [104]:
status = parseReply(reply)
status

'HTTP/1.1 200 OK\r\n'

In [105]:
header = parseHeaders(reply)
header

{'Date': 'Fri, 22 Oct 2021 03:00:24 GMT',
 'Server': 'Apache/2.4.6 (CentOS)',
 'Last-Modified': 'Wed, 16 Dec 2020 23:45:42 GMT',
 'ETag': '"10d-5b69d7922d580"',
 'Accept-Ranges': 'bytes',
 'Content-Length': '269',
 'Connection': 'close',
 'Content-Type': 'application/json'}

In [106]:
body = parseBody(reply)
body

'{"FRA": {"2007": {"pop": 64.02, "gdp": 2657.21}, "2017": {"pop": 66.87, "gdp": 2586.29}}, "GBR": {"2007": {"pop": 61.32, "gdp": 3084.12}, "2017": {"pop": 66.06, "gdp": 2637.87}}, "USA": {"2007": {"pop": 301.23, "gdp": 14451.9}, "2017": {"pop": 325.15, "gdp": 19485.4}}}'